In [13]:
import pickle
import dlib
import numpy as np
from math import cos, sin
import cv2

In [14]:
def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 100):
    
    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [15]:
yaw_model = pickle.load(open(r"C:\Users\AYA\Downloads\yaw_model.sav", 'rb'))
roll_model = pickle.load(open(r"C:\Users\AYA\Downloads\roll_model.sav", 'rb'))
pitch_model= pickle.load(open(r"C:\Users\AYA\Downloads\pitch_model.sav", 'rb'))

In [16]:
pca_yaw = pickle.load(open(r"C:\Users\AYA\Downloads\pca_yaw.pkl", 'rb'))
pca_pitch = pickle.load(open(r"C:\Users\AYA\Downloads\pca_pitch.pkl", 'rb'))
pca_roll = pickle.load(open(r"C:\Users\AYA\Downloads\pca_roll.pkl", 'rb'))

In [17]:
predictor=dlib.shape_predictor(r"C:\Users\AYA\Downloads\shape_predictor_68_face_landmarks.dat\shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

In [21]:
video = cv2.VideoCapture(0)
if (video.isOpened() == False): 
    print("Error reading video file")

frame_width = int(video.get(3))
frame_height = int(video.get(4))
   
size = (frame_width, frame_height)
   
result = cv2.VideoWriter(r'C:\Users\AYA\Downloads\pose_video.avi', cv2.VideoWriter_fourcc(*'MJPG'),10, size)
    
while(True):
    ret, frame = video.read()
  
    if ret == True: 
   
        cv2_frame = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    
        coords = np.zeros((68, 2), dtype='int')
        rects = detector(cv2_frame, 1)
        for (i, rect) in enumerate(rects):
            shape = predictor(cv2_frame, rect)
            for i in range(0, 68):
                  coords[i] = (shape.part(i).x, shape.part(i).y)
                  
        one_sample_yaw = pca_yaw.transform(  coords.reshape(1,136) )
        one_sample_pitch = pca_pitch.transform(  coords.reshape(1,136) )
        one_sample_roll = pca_roll.transform(  coords.reshape(1,136) )
        
        yaw=yaw_model.predict(one_sample_yaw)[0]
        pitch=pitch_model.predict(one_sample_pitch)[0]
        roll=roll_model.predict(one_sample_roll)[0]
        tdx,tdy=0,0        
        for (idx, det) in enumerate(rects):
            print(idx,det , " just")
            # Get x_min, y_min, x_max, y_max, conf
            x_min = det.left()
            y_min = det.top()
            x_max = det.right()-x_min
            y_max = det.bottom()-y_min
            shape = predictor(cv2_frame, det)
            tdx,tdy=shape.part(30).x, shape.part(30).y
        frame=draw_axis(frame,yaw,pitch,roll ,tdx,tdy ) 
        

        result.write(frame)

        
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
  
    # Break the loop
    else:
        break
  

video.release()
result.release()
    
cv2.destroyAllWindows()
   

c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


0 [(304, 201) (489, 387)]  just


c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


0 [(304, 201) (489, 387)]  just


c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


...

0 [(324, 201) (510, 387)]  just


c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


0 [(322, 236) (476, 391)]  just


c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
c:\Users\AYA\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


0 [(304, 222) (489, 407)]  just
